# Chapter 13: Agent Memory Systems
**From: Zero to AI Agent**

## Overview
In this chapter, you'll learn about:
- Types of memory (short-term vs. long-term)
- Conversation memory implementation
- Summary memory for long conversations
- Entity memory and knowledge graphs
- Vector databases for semantic memory
- Implementing memory in LangChain
- Memory management strategies


In [ ]:
!pip install -q -r requirements.txt

from dotenv import load_dotenv
load_dotenv()

---
## Section 13.1: Types of memory (short-term vs. long-term)

In [ ]:
# Section 13.1 content
# No source files found for this section

---
### Section 13.1 Exercises

### Exercise 13.1.1: Memory Classification

For each scenario below, identify whether the information would best be stored in short-term memory, long-term memory, or both. Explain your reasoning.

1. The user says: "Let's talk about my upcoming trip to Paris."
2. The user mentions: "I'm allergic to peanuts."
3. The user asks: "What did I just say about the hotel?"
4. The agent calculates an intermediate result while solving a math problem.
5. The user says: "Remember, I prefer bullet points over long paragraphs."
6. The user shares: "Today's weather is really nice."

In [ ]:
# Your code here


### Exercise 13.1.2: Design a Memory Schema

You're building a personal finance assistant agent. Design a memory schema that specifies:

1. **What to store in short-term memory** during a conversation about budgeting
2. **What to store in long-term memory** across sessions
3. **How information might move** from short-term to long-term storage
4. **What should probably NOT be stored** (and why)

Write out your schema as a structured outline or diagram. Think about data types, categories, and how you'd organize the information.

In [ ]:
# Your code here


### Exercise 13.1.3: Memory Retrieval Strategy

Consider this scenario: You have a personal assistant agent with long-term memory containing hundreds of stored facts about a user. The user asks: "What restaurant should I try this weekend?"

Design a retrieval strategy that answers:

1. What memory categories might be relevant to this question?
2. How would you decide which specific memories to retrieve? (You can't load them all into context)
3. How would you handle conflicting or outdated information? (e.g., "User said they love sushi" from 2 years ago vs. "User mentioned trying to eat less fish" from last month)
4. How would you format the retrieved memories for the LLM to use effectively?

Write out your strategy as a step-by-step algorithm or flowchart, with explanations for each decision.

In [ ]:
# Your code here


---
## Section 13.2: Conversation memory implementation

In [ ]:
# From: no_memory_demo.py

# From: AI Agents Book - Chapter 13, Section 13.2
# File: no_memory_demo.py

from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI

client = OpenAI()

def chat(user_message):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": user_message}
        ]
    )
    return response.choices[0].message.content

# Conversation attempt
print(chat("Hi! My name is Alex."))
# "Hello Alex! Nice to meet you!"

print(chat("What's my name?"))
# "I don't know your name. Could you tell me?"


In [ ]:
# From: simple_memory.py

# From: AI Agents Book - Chapter 13, Section 13.2
# File: simple_memory.py

from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI

client = OpenAI()
conversation_history = []

def chat(user_message):
    # Add the user's message to history
    conversation_history.append({
        "role": "user",
        "content": user_message
    })
    
    # Send entire history to the API
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=conversation_history
    )
    
    # Extract and store the assistant's reply
    assistant_message = response.choices[0].message.content
    conversation_history.append({
        "role": "assistant",
        "content": assistant_message
    })
    
    return assistant_message

# Now let's try again
print(chat("Hi! My name is Alex."))
# "Hello Alex! Nice to meet you! How can I help you today?"

print(chat("What's my name?"))
# "Your name is Alex!"


In [ ]:
# From: conversation_memory.py

# From: AI Agents Book - Chapter 13, Section 13.2
# File: conversation_memory.py

from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI
from datetime import datetime

class ConversationMemory:
    def __init__(self, system_prompt=None, max_messages=50):
        self.client = OpenAI()
        self.max_messages = max_messages
        self.messages = []
        
        if system_prompt:
            self.messages.append({
                "role": "system",
                "content": system_prompt
            })
    
    def add_user_message(self, content):
        self.messages.append({
            "role": "user",
            "content": content,
            "timestamp": datetime.now().isoformat()
        })
        self._trim_if_needed()
    
    def add_assistant_message(self, content):
        self.messages.append({
            "role": "assistant", 
            "content": content,
            "timestamp": datetime.now().isoformat()
        })
    
    def _trim_if_needed(self):
        """Remove oldest messages if we exceed max_messages."""
        while len(self.messages) > self.max_messages:
            # Find first non-system message and remove it
            for i, msg in enumerate(self.messages):
                if msg["role"] != "system":
                    self.messages.pop(i)
                    break
    
    def get_messages_for_api(self):
        """Return messages formatted for API call (without timestamps)."""
        return [
            {"role": m["role"], "content": m["content"]} 
            for m in self.messages
        ]
    
    def chat(self, user_input):
        self.add_user_message(user_input)
        
        response = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=self.get_messages_for_api()
        )
        
        assistant_reply = response.choices[0].message.content
        self.add_assistant_message(assistant_reply)
        
        return assistant_reply
    
    def get_history(self):
        """Return conversation history for inspection."""
        return self.messages.copy()
    
    def clear(self):
        """Clear conversation but keep system prompt."""
        system_msgs = [m for m in self.messages if m["role"] == "system"]
        self.messages = system_msgs


# Example usage
if __name__ == "__main__":
    # Create a memory-enabled conversation
    memory = ConversationMemory(
        system_prompt="You are a friendly travel advisor.",
        max_messages=30
    )

    # Chat naturally
    print(memory.chat("I'm planning a trip to Japan."))
    print(memory.chat("What's the best time to visit?"))
    print(memory.chat("And what about the trip I mentioned?"))  # It remembers!

    # Check history if needed
    for msg in memory.get_history():
        print(f"{msg['role']}: {msg['content'][:50]}...")


In [ ]:
# From: conversation_manager.py

# From: AI Agents Book - Chapter 13, Section 13.2
# File: conversation_manager.py

from conversation_memory import ConversationMemory

class ConversationManager:
    def __init__(self, system_prompt=None):
        self.conversations = {}  # user_id -> ConversationMemory
        self.system_prompt = system_prompt
    
    def get_or_create(self, user_id):
        """Get existing conversation or create new one."""
        if user_id not in self.conversations:
            self.conversations[user_id] = ConversationMemory(
                system_prompt=self.system_prompt
            )
        return self.conversations[user_id]
    
    def chat(self, user_id, message):
        """Chat with a specific user's conversation."""
        memory = self.get_or_create(user_id)
        return memory.chat(message)
    
    def clear_conversation(self, user_id):
        """Clear a specific user's history."""
        if user_id in self.conversations:
            self.conversations[user_id].clear()


# Example usage
if __name__ == "__main__":
    manager = ConversationManager(system_prompt="You are a helpful assistant.")

    # Different users, different conversations
    manager.chat("user_123", "My name is Alice")
    manager.chat("user_456", "My name is Bob")

    print(manager.chat("user_123", "What's my name?"))  # "Alice"
    print(manager.chat("user_456", "What's my name?"))  # "Bob"


In [ ]:
# From: persistent_memory.py

# From: AI Agents Book - Chapter 13, Section 13.2
# File: persistent_memory.py

import json
from pathlib import Path
from conversation_memory import ConversationMemory

class PersistentConversationMemory(ConversationMemory):
    def __init__(self, user_id, storage_dir="conversations", **kwargs):
        super().__init__(**kwargs)
        self.user_id = user_id
        self.storage_path = Path(storage_dir) / f"{user_id}.json"
        self.storage_path.parent.mkdir(exist_ok=True)
        self._load()
    
    def _load(self):
        """Load conversation from disk if it exists."""
        if self.storage_path.exists():
            with open(self.storage_path, 'r') as f:
                data = json.load(f)
                self.messages = data.get("messages", [])
    
    def _save(self):
        """Save conversation to disk."""
        with open(self.storage_path, 'w') as f:
            json.dump({"messages": self.messages}, f, indent=2)
    
    def add_user_message(self, content):
        super().add_user_message(content)
        self._save()
    
    def add_assistant_message(self, content):
        super().add_assistant_message(content)
        self._save()


# Example usage
if __name__ == "__main__":
    # First run - creates new conversation
    memory = PersistentConversationMemory(
        user_id="alice",
        system_prompt="You are a helpful assistant."
    )
    
    print(memory.chat("Hi! I'm planning a vacation."))
    print(f"Saved to: {memory.storage_path}")
    
    # Second run - would load existing conversation
    # memory2 = PersistentConversationMemory(user_id="alice")
    # print(memory2.chat("What was I planning?"))  # It remembers!


---
### Section 13.2 Exercises

### Exercise 13.2.1: Basic Memory Implementation

Create a simple conversation memory system that:
1. Stores messages in a list
2. Includes a system prompt: "You are a helpful math tutor."
3. Has a `chat()` function that maintains history
4. Prints the total message count after each exchange

Test it with this sequence:
- "What is 5 + 3?"
- "Now multiply that by 2"
- "What were we calculating?"

In [ ]:
# Your code here


### Exercise 13.2.2: Smart Trimming

Extend the basic memory system to implement token-aware trimming:
1. Count tokens using a simple approximation (words × 1.3)
2. Set a maximum token limit of 500 tokens
3. When trimming, keep the system prompt and most recent messages
4. Print a warning when trimming occurs

Test with a conversation that would exceed 500 tokens to verify trimming works.

In [ ]:
# Your code here


### Exercise 13.2.3: Conversation Analytics

Build a ConversationMemory class that includes analytics:
1. Track message counts by role (user vs assistant)
2. Track average message length
3. Store timestamps and calculate conversation duration
4. Identify the longest message in the conversation
5. Provide a `get_stats()` method returning all analytics

Create a sample conversation and display the statistics.

In [ ]:
# Your code here


---
## Section 13.3: Summary memory for long conversations

In [ ]:
# From: summary_memory.py

# From: AI Agents Book - Chapter 13, Section 13.3
# File: summary_memory.py

from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI


class SummaryMemory:
    def __init__(self, system_prompt=None, max_messages=30, keep_recent=10):
        self.client = OpenAI()
        self.max_messages = max_messages
        self.keep_recent = keep_recent
        self.messages = []
        self.summaries = []  # Track all summaries for debugging
        
        if system_prompt:
            self.messages.append({
                "role": "system",
                "content": system_prompt
            })
    
    def _count_non_system(self):
        return len([m for m in self.messages if m["role"] != "system"])
    
    def _should_summarize(self):
        return self._count_non_system() > self.max_messages
    
    def _generate_summary(self, messages_to_summarize):
        conversation_text = "\n".join(
            f"{m['role'].upper()}: {m['content']}" 
            for m in messages_to_summarize
        )
        
        response = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{
                "role": "user",
                "content": f"""Summarize this conversation concisely, keeping key facts, 
decisions, and context needed to continue the discussion:

{conversation_text}

Provide a clear, structured summary:"""
            }],
            max_tokens=400
        )
        return response.choices[0].message.content
    
    def _perform_summarization(self):
        # Separate system messages from conversation
        system_msgs = [m for m in self.messages if m["role"] == "system" 
                       and "Summary of earlier" not in m["content"]]
        conversation = [m for m in self.messages if m["role"] != "system"
                        or "Summary of earlier" in m["content"]]
        
        # Select what to summarize vs keep
        to_summarize = conversation[:-self.keep_recent]
        to_keep = conversation[-self.keep_recent:]
        
        # Generate summary
        summary = self._generate_summary(to_summarize)
        self.summaries.append(summary)
        
        # Rebuild messages
        summary_msg = {
            "role": "system",
            "content": f"Summary of earlier conversation:\n{summary}"
        }
        self.messages = system_msgs + [summary_msg] + to_keep
        
        print(f"📝 Summarized {len(to_summarize)} messages into {len(summary.split())} words")
    
    def chat(self, user_input):
        # Add user message
        self.messages.append({"role": "user", "content": user_input})
        
        # Check if summarization needed
        if self._should_summarize():
            self._perform_summarization()
        
        # Make API call
        response = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=self.messages
        )
        
        assistant_reply = response.choices[0].message.content
        self.messages.append({"role": "assistant", "content": assistant_reply})
        
        return assistant_reply
    
    def get_message_count(self):
        return len(self.messages)
    
    def get_summaries(self):
        return self.summaries


# Example usage
if __name__ == "__main__":
    assistant = SummaryMemory(
        system_prompt="""You are a project planning assistant. Help users plan 
and track their projects. Remember details about their projects, team members, 
deadlines, and constraints.""",
        max_messages=20,
        keep_recent=8
    )
    
    # Test conversation
    test_messages = [
        "I'm starting a new mobile app project.",
        "The deadline is March 15th.",
        "My team has 2 developers and 1 designer.",
        "What should we focus on first?",
    ]
    
    for msg in test_messages:
        print(f"User: {msg}")
        response = assistant.chat(msg)
        print(f"Assistant: {response}\n")


In [ ]:
# From: hybrid_memory.py

# From: AI Agents Book - Chapter 13, Section 13.3
# File: hybrid_memory.py

from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI


class HybridMemory:
    """
    Combines multiple memory strategies:
    - Full history for persistence/auditing
    - Active context for LLM (with summarization)
    - Extracted key facts for quick reference
    """
    
    def __init__(self, system_prompt=None, max_active_messages=30):
        self.client = OpenAI()
        self.full_history = []          # Complete record (for persistence)
        self.active_context = []        # What we send to LLM
        self.summaries = []             # Generated summaries
        self.key_facts = {}             # Extracted important facts
        self.max_active = max_active_messages
        
        if system_prompt:
            msg = {"role": "system", "content": system_prompt}
            self.full_history.append(msg)
            self.active_context.append(msg)
    
    def _extract_facts(self, content):
        """Extract key facts from content (placeholder for entity extraction)."""
        # In a full implementation, this would use NLP or LLM to extract
        # entities, preferences, and key information
        # See section 13.4 for full entity extraction
        pass
    
    def _should_summarize(self):
        non_system = [m for m in self.active_context if m["role"] != "system"]
        return len(non_system) > self.max_active
    
    def _summarize(self):
        """Summarize older messages in active context."""
        system_msgs = [m for m in self.active_context 
                       if m["role"] == "system" and "Summary" not in m["content"]]
        conversation = [m for m in self.active_context if m["role"] != "system"]
        
        # Keep recent messages
        keep_recent = 10
        to_summarize = conversation[:-keep_recent]
        to_keep = conversation[-keep_recent:]
        
        if not to_summarize:
            return
        
        # Generate summary
        text = "\n".join(f"{m['role']}: {m['content']}" for m in to_summarize)
        response = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{
                "role": "user",
                "content": f"Summarize concisely:\n\n{text}"
            }],
            max_tokens=300
        )
        summary = response.choices[0].message.content
        self.summaries.append(summary)
        
        # Rebuild active context
        summary_msg = {"role": "system", "content": f"Earlier conversation summary:\n{summary}"}
        self.active_context = system_msgs + [summary_msg] + to_keep
    
    def _trim_if_needed(self):
        """Final safety trim if still too long after summarization."""
        max_total = self.max_active + 15  # Buffer for summaries
        while len(self.active_context) > max_total:
            # Remove oldest non-system message
            for i, m in enumerate(self.active_context):
                if m["role"] != "system":
                    self.active_context.pop(i)
                    break
    
    def process_message(self, role, content):
        """Process and store a message."""
        msg = {"role": role, "content": content}
        
        # Always store full history
        self.full_history.append(msg)
        self.active_context.append(msg)
        
        # Extract key facts
        self._extract_facts(content)
        
        # Summarize if needed
        if self._should_summarize():
            self._summarize()
        
        # Token-trim if still too long
        self._trim_if_needed()
    
    def chat(self, user_input):
        """Chat with the assistant."""
        self.process_message("user", user_input)
        
        response = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=self.active_context
        )
        
        reply = response.choices[0].message.content
        self.process_message("assistant", reply)
        
        return reply
    
    def get_full_history(self):
        """Return complete conversation history."""
        return self.full_history
    
    def get_active_context_size(self):
        """Return current active context size."""
        return len(self.active_context)
    
    def get_key_facts(self):
        """Return extracted key facts."""
        return self.key_facts


# Example usage
if __name__ == "__main__":
    memory = HybridMemory(
        system_prompt="You are a helpful assistant.",
        max_active_messages=15
    )
    
    print(memory.chat("My name is Alice and I love hiking."))
    print(f"Active context: {memory.get_active_context_size()} messages")
    print(f"Full history: {len(memory.get_full_history())} messages")


---
### Section 13.3 Exercises

### Exercise 13.3.1: Basic Summarization

Write a function `summarize_messages(messages)` that:
1. Takes a list of message dictionaries (role/content)
2. Formats them into readable text
3. Uses an LLM to generate a summary
4. Returns the summary string

Test it with a sample 5-message conversation about planning a vacation.

In [ ]:
# Your code here


### Exercise 13.3.2: Triggered Summarization

Build a `SmartMemory` class that:
1. Tracks messages normally
2. Automatically triggers summarization when message count exceeds 15
3. Keeps the 5 most recent messages intact
4. Stores the summary as a system message
5. Prints a notification when summarization occurs

Test with a conversation that grows past the threshold.

In [ ]:
# Your code here


### Exercise 13.3.3: Domain-Specific Summaries

Create a summarization system for a **medical consultation assistant** that:
1. Uses a specialized summary prompt that extracts:
   - Symptoms mentioned
   - Duration of symptoms
   - Medications discussed
   - Recommendations given
   - Follow-up items
2. Structures the summary in a specific format (not free-form prose)
3. Validates that key medical information isn't lost
4. Includes a `get_medical_summary()` method returning structured data

Test with a mock medical consultation conversation.

In [ ]:
# Your code here


---
## Section 13.4: Entity memory and knowledge graphs

In [ ]:
# From: entity_extraction.py

# From: AI Agents Book - Chapter 13, Section 13.4
# File: entity_extraction.py

from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI
import json


def extract_entities(message, client):
    """Extract entities (people, orgs, projects, locations) from a message."""
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{
            "role": "user",
            "content": f"""Extract entities from this message. Return JSON:
{{
    "entities": [
        {{"name": "...", "type": "person|org|project|location|concept", "info": "..."}}
    ]
}}

Message: {message}

Return ONLY valid JSON:"""
        }],
        max_tokens=300
    )
    
    try:
        return json.loads(response.choices[0].message.content)["entities"]
    except:
        return []


# Example usage
if __name__ == "__main__":
    client = OpenAI()
    
    test_message = "Tell Sarah Chen to schedule a meeting with the Globex team about the Q4 launch."
    entities = extract_entities(test_message, client)
    
    print("Extracted entities:")
    for e in entities:
        print(f"  - {e['name']} ({e['type']}): {e.get('info', 'N/A')}")


In [ ]:
# From: entity_memory.py

# From: AI Agents Book - Chapter 13, Section 13.4
# File: entity_memory.py

from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI
from datetime import datetime


class EntityMemory:
    """Store and retrieve entity information from conversations."""
    
    def __init__(self):
        self.entities = {}  # name -> entity data
        self.client = OpenAI()
    
    def update_entity(self, name, entity_type, new_info):
        """Add or update an entity with new information."""
        name_key = name.lower().strip()
        
        if name_key not in self.entities:
            self.entities[name_key] = {
                "name": name,
                "type": entity_type,
                "facts": [],
                "first_seen": datetime.now().isoformat(),
                "mention_count": 0
            }
        
        entity = self.entities[name_key]
        entity["mention_count"] += 1
        entity["last_seen"] = datetime.now().isoformat()
        
        # Add new fact if not duplicate
        if new_info and new_info not in entity["facts"]:
            entity["facts"].append(new_info)
    
    def get_entity(self, name):
        """Retrieve entity by name."""
        return self.entities.get(name.lower().strip())
    
    def get_relevant_entities(self, message):
        """Find entities mentioned in a message."""
        relevant = []
        message_lower = message.lower()
        
        for key, entity in self.entities.items():
            if key in message_lower or entity["name"].lower() in message_lower:
                relevant.append(entity)
        
        return relevant
    
    def format_for_context(self, entities):
        """Format entities for injection into LLM context."""
        if not entities:
            return ""
        
        lines = ["Relevant information about mentioned entities:"]
        for e in entities:
            facts = "; ".join(e["facts"][-5:])  # Last 5 facts
            lines.append(f"- {e['name']} ({e['type']}): {facts}")
        
        return "\n".join(lines)
    
    def get_all_entities(self):
        """Return all stored entities."""
        return list(self.entities.values())


# Example usage
if __name__ == "__main__":
    memory = EntityMemory()
    
    # Add some entities
    memory.update_entity("Sarah Chen", "person", "Account Manager at Acme")
    memory.update_entity("Sarah Chen", "person", "prefers Slack over email")
    memory.update_entity("Globex", "org", "major client")
    memory.update_entity("Q4 Launch", "project", "deadline is December 15")
    
    # Retrieve
    print("Sarah's info:", memory.get_entity("sarah chen"))
    
    # Find relevant entities in a message
    relevant = memory.get_relevant_entities("What's the status of Q4 Launch?")
    print("\nRelevant entities:")
    print(memory.format_for_context(relevant))


In [ ]:
# From: entity_aware_agent.py

# From: AI Agents Book - Chapter 13, Section 13.4
# File: entity_aware_agent.py

from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI
from entity_memory import EntityMemory
from entity_extraction import extract_entities


class EntityAwareAgent:
    """An agent that automatically learns and recalls entity information."""
    
    def __init__(self, system_prompt):
        self.client = OpenAI()
        self.entity_memory = EntityMemory()
        self.conversation = [{"role": "system", "content": system_prompt}]
    
    def process_message(self, user_message):
        # 1. RETRIEVE: Find relevant entities
        relevant = self.entity_memory.get_relevant_entities(user_message)
        entity_context = self.entity_memory.format_for_context(relevant)
        
        # 2. BUILD CONTEXT: Inject entity knowledge
        messages = self.conversation.copy()
        if entity_context:
            messages.insert(1, {
                "role": "system", 
                "content": entity_context
            })
        messages.append({"role": "user", "content": user_message})
        
        # 3. REASON & ACT: Get agent response
        response = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages
        )
        reply = response.choices[0].message.content
        
        # 4. STORE: Extract and save new entities
        self._extract_and_store(user_message)
        self._extract_and_store(reply)
        
        # Update conversation
        self.conversation.append({"role": "user", "content": user_message})
        self.conversation.append({"role": "assistant", "content": reply})
        
        return reply
    
    def _extract_and_store(self, text):
        """Extract entities from text and update memory."""
        entities = extract_entities(text, self.client)
        for e in entities:
            self.entity_memory.update_entity(
                e["name"], e["type"], e.get("info", "")
            )
    
    def get_known_entities(self):
        """Return all entities the agent knows about."""
        return self.entity_memory.get_all_entities()


# Example usage
if __name__ == "__main__":
    agent = EntityAwareAgent(
        system_prompt="You are a helpful project management assistant."
    )
    
    # Have a conversation
    print("User: I'm working with Sarah Chen on the Q4 Launch project.")
    response = agent.process_message("I'm working with Sarah Chen on the Q4 Launch project.")
    print(f"Agent: {response}\n")
    
    print("User: Sarah said the deadline is December 15th.")
    response = agent.process_message("Sarah said the deadline is December 15th.")
    print(f"Agent: {response}\n")
    
    # Show what the agent learned
    print("Known entities:")
    for entity in agent.get_known_entities():
        print(f"  - {entity['name']} ({entity['type']}): {entity['facts']}")


In [ ]:
# From: knowledge_graph.py

# From: AI Agents Book - Chapter 13, Section 13.4
# File: knowledge_graph.py

from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI
from datetime import datetime
import json


class KnowledgeGraph:
    """Store entities and their relationships."""
    
    def __init__(self):
        self.entities = {}      # name -> attributes
        self.relationships = [] # (entity1, relation, entity2)
    
    def add_entity(self, name, entity_type, attributes=None):
        """Add or update an entity."""
        key = name.lower()
        if key not in self.entities:
            self.entities[key] = {
                "name": name,
                "type": entity_type,
                "attributes": attributes or {}
            }
        elif attributes:
            self.entities[key]["attributes"].update(attributes)
    
    def add_relationship(self, entity1, relation, entity2):
        """Add a relationship between entities."""
        self.relationships.append({
            "from": entity1.lower(),
            "relation": relation,
            "to": entity2.lower(),
            "added": datetime.now().isoformat()
        })
    
    def get_connections(self, entity_name):
        """Get all relationships involving an entity."""
        name = entity_name.lower()
        connections = []
        
        for rel in self.relationships:
            if rel["from"] == name:
                connections.append(f"{rel['relation']} {rel['to']}")
            elif rel["to"] == name:
                connections.append(f"{rel['from']} {rel['relation']} this")
        
        return connections
    
    def extract_relationships(self, message, client):
        """Use LLM to extract relationships from text."""
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{
                "role": "user",
                "content": f"""Extract relationships from this message.
Return JSON: {{"relationships": [{{"from": "...", "relation": "...", "to": "..."}}]}}

Examples of relations: works_on, manages, reports_to, is_part_of, depends_on, located_in

Message: {message}
JSON:"""
            }],
            max_tokens=200
        )
        
        try:
            data = json.loads(response.choices[0].message.content)
            for rel in data.get("relationships", []):
                self.add_relationship(rel["from"], rel["relation"], rel["to"])
        except:
            pass
    
    def format_graph(self):
        """Return a formatted string representation of the graph."""
        lines = ["Entities:"]
        for key, entity in self.entities.items():
            lines.append(f"  - {entity['name']} ({entity['type']})")
        
        lines.append("\nRelationships:")
        for rel in self.relationships:
            lines.append(f"  - {rel['from']} --{rel['relation']}--> {rel['to']}")
        
        return "\n".join(lines)


# Example usage
if __name__ == "__main__":
    client = OpenAI()
    graph = KnowledgeGraph()
    
    # Add entities
    graph.add_entity("Sarah Chen", "person", {"role": "Engineer"})
    graph.add_entity("Q4 Launch", "project")
    graph.add_entity("Acme Corp", "org")
    
    # Add relationships manually
    graph.add_relationship("Sarah Chen", "works_on", "Q4 Launch")
    graph.add_relationship("Q4 Launch", "is_for", "Acme Corp")
    
    # Extract relationships from text
    graph.extract_relationships(
        "The marketing team reports to Jennifer, who manages the NYC office.",
        client
    )
    
    print(graph.format_graph())
    print("\nConnections for Sarah Chen:", graph.get_connections("Sarah Chen"))


In [ ]:
# From: context_aware_agent.py

# From: AI Agents Book - Chapter 13, Section 13.4
# File: context_aware_agent.py

from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI
from entity_memory import EntityMemory
from knowledge_graph import KnowledgeGraph
from entity_extraction import extract_entities


class ContextAwareAgent:
    """
    Agent pattern combining conversation history, entity memory, and knowledge graph.
    This forms the backbone of production AI assistants.
    """
    
    def __init__(self, name, system_prompt):
        self.name = name
        self.client = OpenAI()
        self.system_prompt = system_prompt
        
        # Memory systems
        self.conversation = []
        self.entities = EntityMemory()
        self.graph = KnowledgeGraph()
    
    def build_context(self, user_message):
        """Assemble full context for the agent."""
        # Start with system prompt
        messages = [{"role": "system", "content": self.system_prompt}]
        
        # Add entity context
        relevant = self.entities.get_relevant_entities(user_message)
        if relevant:
            entity_info = self.entities.format_for_context(relevant)
            messages.append({"role": "system", "content": entity_info})
        
        # Add conversation history (last N messages)
        messages.extend(self.conversation[-20:])
        
        # Add current message
        messages.append({"role": "user", "content": user_message})
        
        return messages
    
    def run(self, user_message):
        """Main agent loop."""
        # Build context with entity knowledge
        messages = self.build_context(user_message)
        
        # Get response
        response = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages
        )
        reply = response.choices[0].message.content
        
        # Learn from this exchange
        self._learn(user_message)
        self._learn(reply)
        
        # Update conversation
        self.conversation.append({"role": "user", "content": user_message})
        self.conversation.append({"role": "assistant", "content": reply})
        
        return reply
    
    def _learn(self, text):
        """Extract entities and relationships from text."""
        # Extract entities
        entities = extract_entities(text, self.client)
        for e in entities:
            self.entities.update_entity(e["name"], e["type"], e.get("info"))
        
        # Extract relationships
        self.graph.extract_relationships(text, self.client)
    
    def get_memory_stats(self):
        """Return statistics about what the agent has learned."""
        return {
            "conversation_messages": len(self.conversation),
            "known_entities": len(self.entities.entities),
            "relationships": len(self.graph.relationships)
        }


# Example usage
if __name__ == "__main__":
    agent = ContextAwareAgent(
        name="ProjectBot",
        system_prompt="You are a helpful project management assistant. Track people, projects, and deadlines."
    )
    
    # Simulate a conversation
    messages = [
        "I'm starting a new project called Phoenix with my team lead Maria.",
        "Maria said the deadline is end of Q1. She's also working with Tom on design.",
        "What do you know about the Phoenix project so far?",
        "Who is working on Phoenix?"
    ]
    
    for msg in messages:
        print(f"User: {msg}")
        response = agent.run(msg)
        print(f"Agent: {response}\n")
    
    # Show what was learned
    print("=" * 50)
    print("Memory Stats:", agent.get_memory_stats())
    print("\nKnown Entities:")
    for entity in agent.entities.get_all_entities():
        print(f"  - {entity['name']} ({entity['type']})")


---
### Section 13.4 Exercises

### Exercise 13.4.1: Basic Entity Extraction

Write a function that extracts entities from a message and prints them. Test with: "John from marketing wants to discuss the Phoenix project with the Tokyo team next Tuesday."

In [ ]:
# Your code here


### Exercise 13.4.2: Entity Memory Class

Create an `EntityMemory` class that:
1. Stores entities with names, types, and facts
2. Updates existing entities with new information
3. Retrieves entities by name
4. Formats relevant entities for LLM context

Test by processing 3-4 messages that mention overlapping entities.

In [ ]:
# Your code here


### Exercise 13.4.3: Entity-Aware Agent

Build a simple agent that:
1. Maintains conversation history
2. Extracts and stores entities from each exchange
3. Retrieves relevant entity context before responding
4. Shows what entities it knows when asked "What do you know about [name]?"

Keep the implementation focused—under 80 lines total.

In [ ]:
# Your code here


---
## Section 13.5: Vector databases for semantic memory

In [ ]:
# From: get_embeddings.py

# From: AI Agents Book - Chapter 13, Section 13.5
# File: get_embeddings.py

from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI
import numpy as np

client = OpenAI()


def get_embedding(text):
    """Convert text to a vector embedding."""
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=text
    )
    return response.data[0].embedding


def cosine_similarity(v1, v2):
    """Calculate cosine similarity between two vectors."""
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


# Example usage
if __name__ == "__main__":
    # Get embeddings for test sentences
    vec1 = get_embedding("I love programming")
    vec2 = get_embedding("Coding is my passion")
    vec3 = get_embedding("The weather is nice today")
    
    print(f"Vector length: {len(vec1)}")  # 1536 dimensions
    
    # Calculate similarities
    print(f"'programming' vs 'coding': {cosine_similarity(vec1, vec2):.3f}")  # ~0.85
    print(f"'programming' vs 'weather': {cosine_similarity(vec1, vec3):.3f}")  # ~0.45


In [ ]:
# From: test_chromadb.py

# From: AI Agents Book - Chapter 13, Section 13.5
# File: test_chromadb.py

import chromadb

# Create a client (in-memory by default)
client = chromadb.Client()

# Create a collection (like a table)
collection = client.create_collection("test")

# Add some data
collection.add(
    ids=["id1", "id2"],
    documents=["Hello world", "Goodbye world"]
)

print(f"Collection has {collection.count()} items")
# Output: Collection has 2 items


In [ ]:
# From: semantic_memory.py

# From: AI Agents Book - Chapter 13, Section 13.5
# File: semantic_memory.py

from dotenv import load_dotenv
load_dotenv()

import chromadb
from openai import OpenAI


class SemanticMemory:
    """Vector-based memory for semantic search."""
    
    def __init__(self, collection_name="memories"):
        self.openai = OpenAI()
        self.chroma = chromadb.Client()
        self.collection = self.chroma.create_collection(name=collection_name)
        self.memory_count = 0
    
    def _get_embedding(self, text):
        """Get embedding vector for text."""
        response = self.openai.embeddings.create(
            model="text-embedding-3-small",
            input=text
        )
        return response.data[0].embedding
    
    def add(self, content, metadata=None):
        """Store a memory with its embedding."""
        self.memory_count += 1
        memory_id = f"mem_{self.memory_count}"
        
        self.collection.add(
            ids=[memory_id],
            embeddings=[self._get_embedding(content)],
            documents=[content],
            metadatas=[metadata or {}]
        )
        return memory_id
    
    def search(self, query, n_results=5):
        """Find memories similar to the query."""
        results = self.collection.query(
            query_embeddings=[self._get_embedding(query)],
            n_results=n_results
        )
        
        memories = []
        for i, doc in enumerate(results["documents"][0]):
            memories.append({
                "content": doc,
                "metadata": results["metadatas"][0][i],
                "id": results["ids"][0][i]
            })
        return memories
    
    def count(self):
        """Return total number of memories."""
        return self.collection.count()


# Example usage
if __name__ == "__main__":
    memory = SemanticMemory()
    
    # Add various memories
    memory.add("Sarah is leading the backend team", {"type": "person"})
    memory.add("The project deadline is March 15th", {"type": "project"})
    memory.add("Team velocity has improved by 20%", {"type": "metrics"})
    
    print(f"Total memories: {memory.count()}")
    
    # Search by meaning
    results = memory.search("How is the team performing?")
    print("\nSearch results for 'How is the team performing?':")
    for r in results:
        print(f"  - {r['content']}")


In [ ]:
# From: semantic_agent.py

# From: AI Agents Book - Chapter 13, Section 13.5
# File: semantic_agent.py

from dotenv import load_dotenv
load_dotenv()

import chromadb
from openai import OpenAI
from datetime import datetime


class SemanticMemoryAgent:
    """Agent that uses semantic memory for context retrieval."""
    
    def __init__(self, system_prompt):
        self.openai = OpenAI()
        self.system_prompt = system_prompt
        self.conversation = []
        
        # Set up semantic memory
        self.chroma = chromadb.Client()
        self.memories = self.chroma.create_collection("agent_memories")
        self.memory_id = 0
    
    def _embed(self, text):
        """Get embedding for text."""
        response = self.openai.embeddings.create(
            model="text-embedding-3-small", input=text
        )
        return response.data[0].embedding
    
    def _store_memory(self, content, memory_type="conversation"):
        """Store a piece of information in semantic memory."""
        self.memory_id += 1
        self.memories.add(
            ids=[f"m{self.memory_id}"],
            embeddings=[self._embed(content)],
            documents=[content],
            metadatas=[{"type": memory_type, "timestamp": datetime.now().isoformat()}]
        )
    
    def _retrieve_relevant(self, query, n=3):
        """Retrieve memories relevant to the query."""
        if self.memories.count() == 0:
            return []
        
        results = self.memories.query(
            query_embeddings=[self._embed(query)],
            n_results=min(n, self.memories.count())
        )
        return results["documents"][0] if results["documents"] else []
    
    def chat(self, user_input):
        """Process user input and return response."""
        # 1. Retrieve relevant memories
        relevant = self._retrieve_relevant(user_input)
        
        # 2. Build context
        messages = [{"role": "system", "content": self.system_prompt}]
        
        if relevant:
            memory_context = "Relevant information from memory:\n" + "\n".join(f"- {m}" for m in relevant)
            messages.append({"role": "system", "content": memory_context})
        
        messages.extend(self.conversation[-10:])  # Recent conversation
        messages.append({"role": "user", "content": user_input})
        
        # 3. Generate response
        response = self.openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages
        )
        reply = response.choices[0].message.content
        
        # 4. Update conversation history
        self.conversation.append({"role": "user", "content": user_input})
        self.conversation.append({"role": "assistant", "content": reply})
        
        # 5. Store this exchange in semantic memory
        self._store_memory(f"User said: {user_input}")
        self._store_memory(f"Assistant replied about: {user_input[:50]}")
        
        return reply
    
    def remember(self, fact):
        """Explicitly store a fact in memory."""
        self._store_memory(fact, memory_type="explicit")
        return f"I'll remember: {fact}"


# Example usage
if __name__ == "__main__":
    agent = SemanticMemoryAgent(
        system_prompt="You are a helpful project management assistant."
    )
    
    # Have a conversation
    print("Agent: ", agent.chat("Our project is called Phoenix and it's due March 15th"))
    print("Agent: ", agent.chat("Sarah is the tech lead and Tom handles backend"))
    print("Agent: ", agent.chat("We're worried about the authentication module"))
    
    # Later, ask something related
    print("\nAgent: ", agent.chat("What are the main concerns with our project?"))


In [ ]:
# From: document_memory.py

# From: AI Agents Book - Chapter 13, Section 13.5
# File: document_memory.py

from semantic_memory import SemanticMemory


def chunk_text(text, chunk_size=500, overlap=50):
    """Split text into overlapping chunks."""
    words = text.split()
    chunks = []
    
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i + chunk_size])
        if chunk:
            chunks.append(chunk)
    
    return chunks


class DocumentMemory(SemanticMemory):
    """Semantic memory that handles long documents by chunking."""
    
    def add_document(self, content, source=None):
        """Add a long document by chunking it."""
        chunks = chunk_text(content)
        
        for i, chunk in enumerate(chunks):
            self.add(
                content=chunk,
                metadata={
                    "source": source or "unknown",
                    "chunk_index": i,
                    "total_chunks": len(chunks)
                }
            )
        
        return len(chunks)


# Example usage
if __name__ == "__main__":
    doc_memory = DocumentMemory()
    
    meeting_notes = """
    Project Phoenix kickoff meeting - January 10th.
    Attendees: Sarah (tech lead), Tom (backend), Alex (design), Mike (PM).
    
    We discussed the project timeline. The deadline is March 15th, giving us 
    roughly 10 weeks. Sarah raised concerns about the authentication module
    complexity. Tom suggested using Auth0 to save time.
    
    Budget was confirmed at $50,000. This needs to cover all development 
    and third-party services. Alex will present design mockups next week.
    
    Action items:
    - Sarah: Research Auth0 integration (due Jan 15)
    - Tom: Set up development environment (due Jan 12)
    - Alex: Complete wireframes (due Jan 17)
    - Mike: Create detailed project timeline (due Jan 14)
    """
    
    chunks_added = doc_memory.add_document(meeting_notes, source="kickoff_meeting")
    print(f"Added {chunks_added} chunks")
    
    # Now search across the document
    results = doc_memory.search("What are the concerns about the project?")
    print("\nSearch results:")
    for r in results[:3]:
        print(f"  - {r['content'][:100]}...")


---
### Section 13.5 Exercises

### Exercise 13.5.1: Basic Semantic Search

Create a `SemanticMemory` class that:
1. Stores text with embeddings using ChromaDB
2. Has `add(text)` and `search(query)` methods
3. Returns the top 3 most similar results

Test by adding 5 facts about different topics and searching for related concepts.

In [ ]:
# Your code here


### Exercise 13.5.2: Memory with Categories

Extend your semantic memory to:
1. Store memories with a "category" metadata field
2. Add a `search_category(query, category)` method that filters by category
3. Track how many memories exist per category

Test with memories in categories like "work", "personal", "ideas".

In [ ]:
# Your code here


### Exercise 13.5.3: Conversational Agent with Semantic Recall

Build an agent that:
1. Maintains conversation history
2. Stores each exchange in semantic memory
3. Retrieves relevant past conversations when responding
4. Has a `remember(fact)` method for explicit memory storage
5. Shows retrieved memories in debug output

Test with a multi-turn conversation where later questions relate to earlier topics.

In [ ]:
# Your code here


---
## Section 13.6: Implementing memory in LangChain

In [ ]:
# From: test_langchain_setup.py

# From: AI Agents Book - Chapter 13, Section 13.6
# File: test_langchain_setup.py

from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

llm = ChatOpenAI(model="gpt-3.5-turbo")
response = llm.invoke([HumanMessage(content="Hello!")])
print(response.content)
print("LangChain is ready!")


In [ ]:
# From: basic_memory_chain.py

# From: AI Agents Book - Chapter 13, Section 13.6
# File: basic_memory_chain.py

from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

# Store for session histories (in production, use a database)
store = {}


def get_session_history(session_id: str):
    """Retrieve or create history for a session."""
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


# Create the prompt with a placeholder for history
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

# Create the chain
llm = ChatOpenAI(model="gpt-3.5-turbo")
chain = prompt | llm

# Wrap with message history
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

# Use it with a session ID
config = {"configurable": {"session_id": "user_123"}}

response1 = chain_with_history.invoke(
    {"input": "Hi! My name is Alice."},
    config=config
)
print(response1.content)

response2 = chain_with_history.invoke(
    {"input": "What's my name?"},
    config=config
)
print(response2.content)  # It remembers Alice!


In [ ]:
# From: trimmed_memory.py

# From: AI Agents Book - Chapter 13, Section 13.6
# File: trimmed_memory.py
#
# Demonstrates using trim_messages to limit conversation history size.
# This prevents context window overflow in long conversations.

from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import trim_messages
from langchain_community.chat_message_histories import ChatMessageHistory

store = {}


def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


# Create a message trimmer
trimmer = trim_messages(
    max_tokens=200,  # Keep roughly this many tokens
    strategy="last",  # Keep the most recent messages
    token_counter=ChatOpenAI(model="gpt-3.5-turbo"),  # Use LLM to count
    include_system=True,  # Always keep system message
    start_on="human",  # Start sequence on human message
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Remember details users share with you."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

llm = ChatOpenAI(model="gpt-3.5-turbo")


# Use RunnablePassthrough.assign to trim history before passing to prompt
chain = (
    RunnablePassthrough.assign(
        history=lambda x: trimmer.invoke(x.get("history", []))
    )
    | prompt
    | llm
)

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)


def chat(message: str, session_id: str = "user_456") -> str:
    """Send a message and get a response."""
    config = {"configurable": {"session_id": session_id}}
    response = chain_with_history.invoke({"input": message}, config=config)
    return response.content


# Long conversation - older messages will be trimmed
if __name__ == "__main__":
    messages = [
        "My name is Bob.",
        "I live in Seattle.",
        "I work as a data scientist.",
        "My favorite language is Python.",
        "I have a dog named Max.",
        "I enjoy hiking on weekends.",
        "What do you remember about me?"
    ]
    
    print("=" * 50)
    print("Trimmed Memory Demo (200 token limit)")
    print("=" * 50)
    print()
    
    for msg in messages:
        print(f"Human: {msg}")
        response = chat(msg)
        print(f"AI: {response}")
        print()
    
    print("=" * 50)
    print("With only ~200 tokens, early messages get trimmed.")
    print("The bot may not remember name/city from the start.")
    print("=" * 50)

In [ ]:
# From: summary_memory_modern.py

# From: AI Agents Book - Chapter 13, Section 13.6
# File: summary_memory_modern.py

from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage


class SummaryMemory:
    """Custom summary memory implementation for modern LangChain."""
    
    def __init__(self, llm, max_messages=10):
        self.llm = llm
        self.max_messages = max_messages
        self.messages = []
        self.summary = ""
    
    def add_exchange(self, human_msg: str, ai_msg: str):
        """Add a human-AI exchange to memory."""
        self.messages.append(HumanMessage(content=human_msg))
        self.messages.append(AIMessage(content=ai_msg))
        
        if len(self.messages) > self.max_messages:
            self._summarize()
    
    def _summarize(self):
        """Summarize older messages to compress memory."""
        to_summarize = self.messages[:-4]
        to_keep = self.messages[-4:]
        
        summary_prompt = f"""Summarize concisely, preserving key facts:
Previous: {self.summary}
New: {self._format(to_summarize)}
Updated summary:"""
        
        response = self.llm.invoke([HumanMessage(content=summary_prompt)])
        self.summary = response.content
        self.messages = to_keep
    
    def _format(self, messages):
        """Format messages for summary prompt."""
        return "\n".join(
            f"{'H' if isinstance(m, HumanMessage) else 'A'}: {m.content}" 
            for m in messages
        )
    
    def get_context(self):
        """Get messages to include in context."""
        context = []
        if self.summary:
            context.append(SystemMessage(content=f"Earlier: {self.summary}"))
        context.extend(self.messages)
        return context


# Usage example
if __name__ == "__main__":
    llm = ChatOpenAI(model="gpt-3.5-turbo")
    memory = SummaryMemory(llm, max_messages=6)
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}")
    ])
    
    chain = prompt | llm
    
    test_messages = [
        "I'm planning a Japan trip.",
        "Budget is $5000.",
        "I love temples.",
        "What should I prioritize?"
    ]
    
    for msg in test_messages:
        messages = memory.get_context() + [HumanMessage(content=msg)]
        response = chain.invoke({"history": messages, "input": msg})
        print(f"H: {msg}\nA: {response.content[:80]}...\n")
        memory.add_exchange(msg, response.content)


In [ ]:
# From: persistent_sqlite.py

# From: AI Agents Book - Chapter 13, Section 13.6
# File: persistent_sqlite.py
#
# IMPORTANT: This uses LangChain's built-in SQLChatMessageHistory, which
# creates a schema WITHOUT timestamps. For retention policies and cleanup
# that require timestamps, see retention_policy.py and production_memory_manager.py
# which use a custom schema.
#
# The SQLChatMessageHistory schema:
#   - id (INTEGER PRIMARY KEY)
#   - session_id (TEXT)
#   - message (TEXT - JSON blob)
#
# For timestamp-based retention, use the custom schema in Section 13.7 files.

from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import SQLChatMessageHistory


# Use a unique database name to avoid conflicts with other examples
DB_PATH = "sqlite:///langchain_chat.db"


def get_session_history(session_id: str):
    """Get SQLite-backed message history.
    
    Note: LangChain's SQLChatMessageHistory does not store timestamps.
    For production use with retention policies, consider using a custom
    schema (see retention_policy.py).
    """
    return SQLChatMessageHistory(
        session_id=session_id,
        connection=DB_PATH
    )


prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Remember what the user tells you."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

chain = prompt | ChatOpenAI(model="gpt-3.5-turbo")

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)


def chat(session_id: str, message: str) -> str:
    """Send a message and get a response."""
    config = {"configurable": {"session_id": session_id}}
    response = chain_with_history.invoke({"input": message}, config=config)
    return response.content


if __name__ == "__main__":
    print("=" * 50)
    print("Persistent SQLite Memory Demo")
    print("=" * 50)
    print(f"Database: {DB_PATH}")
    print()
    
    # First interaction
    print("Human: My favorite color is blue.")
    response = chat("alice_session", "My favorite color is blue.")
    print(f"AI: {response}")
    print()
    
    # Second interaction - should remember
    print("Human: What's my favorite color?")
    response = chat("alice_session", "What's my favorite color?")
    print(f"AI: {response}")
    print()
    
    print("=" * 50)
    print("Try stopping and restarting this script.")
    print("The memory will persist in langchain_chat.db!")
    print("=" * 50)

In [ ]:
# From: multiuser_chat.py

# From: AI Agents Book - Chapter 13, Section 13.6
# File: multiuser_chat.py
#
# Demonstrates multi-user support with isolated memory per user.
# Each user_id (session_id) gets completely separate conversation history.
#
# Note: Uses LangChain's SQLChatMessageHistory which does not include
# timestamps. For production with retention policies, see Section 13.7.

from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import SQLChatMessageHistory


# Unique database for multi-user demo
DB_PATH = "sqlite:///multiuser.db"


def get_session_history(session_id: str):
    """Get isolated SQLite history per user.
    
    Each session_id gets completely separate conversation history.
    Users cannot see each other's messages.
    """
    return SQLChatMessageHistory(
        session_id=session_id,
        connection=DB_PATH
    )


prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Remember what each user tells you."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

chain = prompt | ChatOpenAI(model="gpt-3.5-turbo")

chat = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)


def send_message(user_id: str, message: str) -> str:
    """Send a message as a specific user."""
    config = {"configurable": {"session_id": user_id}}
    response = chat.invoke({"input": message}, config=config)
    return response.content


# Different users, different memories
if __name__ == "__main__":
    print("=" * 50)
    print("Multi-User Chat Demo")
    print("=" * 50)
    print(f"Database: {DB_PATH}")
    print()
    
    # Alice and Bob have separate memories
    print("--- Setting up preferences ---")
    print("Alice: I love Python.")
    print(f"  AI: {send_message('alice', 'I love Python.')}")
    print()
    
    print("Bob: I prefer JavaScript.")
    print(f"  AI: {send_message('bob', 'I prefer JavaScript.')}")
    print()
    
    # Each user's memory is isolated
    print("--- Testing memory isolation ---")
    print("Alice: What language do I love?")
    print(f"  AI: {send_message('alice', 'What language do I love?')}")
    print()
    
    print("Bob: What language do I prefer?")
    print(f"  AI: {send_message('bob', 'What language do I prefer?')}")
    print()
    
    print("=" * 50)
    print("Notice: Alice and Bob have separate memories!")
    print("Alice knows about Python, Bob knows about JavaScript.")
    print("=" * 50)

In [ ]:
# From: agent_with_memory.py

# From: AI Agents Book - Chapter 13, Section 13.6
# File: agent_with_memory.py
# Note: This uses modern LangChain patterns. Check langchain docs for latest API.

from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_classic.agents import create_react_agent, AgentExecutor
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.tools import tool

# Define tools
@tool
def calculator(expression: str) -> str:
    """Evaluate a math expression like '2+2' or '10*5'."""
    try:
        return f"Result: {eval(expression)}"
    except:
        return "Error evaluating expression"


@tool
def get_weather(city: str) -> str:
    """Get current weather for a city."""
    weather_data = {"Seattle": "Rainy, 55°F", "NYC": "Sunny, 68°F"}
    return weather_data.get(city, f"Weather in {city}: Partly cloudy, 70°F")


tools = [calculator, get_weather]

# Initialize model
model = ChatOpenAI(model="gpt-4o")

# ReAct prompt template - must include {tools}, {tool_names}, and {agent_scratchpad}
REACT_PROMPT = """You are a helpful assistant with access to tools.

You have access to the following tools:
{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Previous conversation:
{chat_history}

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate.from_template(REACT_PROMPT)

# Create agent
agent = create_react_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Simple conversation with manual history management
chat_history = []


def format_chat_history(history):
    """Format chat history as a string for the prompt."""
    if not history:
        return "No previous conversation."
    
    formatted = []
    for msg in history:
        if isinstance(msg, HumanMessage):
            formatted.append(f"Human: {msg.content}")
        elif isinstance(msg, AIMessage):
            formatted.append(f"Assistant: {msg.content}")
        elif isinstance(msg, str):
            formatted.append(f"Assistant: {msg}")
    return "\n".join(formatted)


def chat(user_input):
    """Chat with the agent, maintaining history."""
    response = agent_executor.invoke({
        "input": user_input,
        "chat_history": format_chat_history(chat_history)
    })
    
    # Update history
    chat_history.append(HumanMessage(content=user_input))
    chat_history.append(AIMessage(content=response["output"]))
    
    return response["output"]


if __name__ == "__main__":
    # First interaction
    print("Agent:", chat("What's the weather in Seattle?"))
    
    # Follow-up - agent should remember
    print("Agent:", chat("What city did I just ask about?"))
    
    # Use a tool
    print("Agent:", chat("Calculate 25 * 4"))

---
### Section 13.6 Exercises

### Exercise 13.6.1: Basic Chat with Memory

Create a chat using `RunnableWithMessageHistory` that:
1. Uses in-memory `ChatMessageHistory`
2. Remembers the user's name across 3 messages
3. Prints the full history at the end

In [ ]:
# Your code here


### Exercise 13.6.2: Windowed Memory

Build a chatbot that:
1. Uses `trim_messages` to keep only ~100 tokens
2. Has a 6-message conversation
3. Demonstrates early messages are forgotten
4. Shows what the bot remembers at the end

In [ ]:
# Your code here


### Exercise 13.6.3: Agent with Memory Management

Create an agent that:
1. Has 2 tools (calculator, weather)
2. Manages memory with automatic summarization
3. Keeps only recent messages after summarizing
4. Demonstrates memory works across tool calls

In [ ]:
# Your code here


---
## Section 13.7: Memory management strategies

In [ ]:
# From: retention_policy.py

# From: AI Agents Book - Chapter 13, Section 13.7
# File: retention_policy.py

from dotenv import load_dotenv
load_dotenv()

from datetime import datetime, timedelta
import sqlite3
import json


class RetentionPolicy:
    """Manage conversation data retention and GDPR compliance.
    
    This class creates its own table schema with timestamps for proper
    retention management. The default LangChain SQLChatMessageHistory
    doesn't include timestamps, so we manage our own schema here.
    """
    
    def __init__(self, db_path="retention_demo.db", retention_days=30):
        self.db_path = db_path
        self.retention_days = retention_days
        self._init_db()
    
    def _init_db(self):
        """Initialize database with timestamp-enabled schema."""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        # Create table with timestamp column
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS message_store (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                session_id TEXT NOT NULL,
                message_type TEXT NOT NULL,
                content TEXT NOT NULL,
                timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
            )
        """)
        
        # Create index for efficient cleanup queries
        cursor.execute("""
            CREATE INDEX IF NOT EXISTS idx_timestamp 
            ON message_store(timestamp)
        """)
        cursor.execute("""
            CREATE INDEX IF NOT EXISTS idx_session_id 
            ON message_store(session_id)
        """)
        
        conn.commit()
        conn.close()
    
    def add_message(self, session_id: str, message_type: str, content: str):
        """Add a message with automatic timestamp."""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute("""
            INSERT INTO message_store (session_id, message_type, content, timestamp)
            VALUES (?, ?, ?, ?)
        """, (session_id, message_type, content, datetime.now()))
        
        conn.commit()
        conn.close()
    
    def cleanup_old_conversations(self):
        """Delete conversations older than retention period."""
        cutoff_date = datetime.now() - timedelta(days=self.retention_days)
        
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        # Delete old messages
        cursor.execute("""
            DELETE FROM message_store 
            WHERE timestamp < ?
        """, (cutoff_date,))
        
        deleted_count = cursor.rowcount
        conn.commit()
        conn.close()
        
        return deleted_count
    
    def get_user_data(self, session_id: str):
        """Retrieve all data for a specific user (GDPR data export)."""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute("""
            SELECT id, session_id, message_type, content, timestamp 
            FROM message_store 
            WHERE session_id = ?
            ORDER BY timestamp
        """, (session_id,))
        
        columns = ['id', 'session_id', 'message_type', 'content', 'timestamp']
        data = [dict(zip(columns, row)) for row in cursor.fetchall()]
        conn.close()
        return data
    
    def export_user_data_json(self, session_id: str) -> str:
        """Export user data as JSON (GDPR compliance)."""
        data = self.get_user_data(session_id)
        # Convert datetime objects to strings
        for record in data:
            if record['timestamp']:
                record['timestamp'] = str(record['timestamp'])
        return json.dumps(data, indent=2)
    
    def delete_user_data(self, session_id: str):
        """Delete all data for a specific user (GDPR right to erasure)."""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute("""
            DELETE FROM message_store 
            WHERE session_id = ?
        """, (session_id,))
        
        deleted_count = cursor.rowcount
        conn.commit()
        conn.close()
        
        return deleted_count
    
    def get_stats(self):
        """Get database statistics."""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute("SELECT COUNT(*) FROM message_store")
        total_messages = cursor.fetchone()[0]
        
        cursor.execute("SELECT COUNT(DISTINCT session_id) FROM message_store")
        total_sessions = cursor.fetchone()[0]
        
        cursor.execute("SELECT MIN(timestamp), MAX(timestamp) FROM message_store")
        date_range = cursor.fetchone()
        
        conn.close()
        
        return {
            "total_messages": total_messages,
            "total_sessions": total_sessions,
            "oldest_message": date_range[0],
            "newest_message": date_range[1]
        }


# Usage
if __name__ == "__main__":
    policy = RetentionPolicy(db_path="retention_demo.db", retention_days=30)
    
    # Add some sample messages
    print("Adding sample messages...")
    policy.add_message("user_123", "human", "Hello, I need help with my account")
    policy.add_message("user_123", "ai", "I'd be happy to help! What do you need?")
    policy.add_message("user_456", "human", "What's the weather like?")
    policy.add_message("user_456", "ai", "I don't have access to weather data.")
    
    # Show stats
    stats = policy.get_stats()
    print(f"\nDatabase stats: {stats}")
    
    # Export user data (GDPR)
    print(f"\nUser data for user_123:")
    print(policy.export_user_data_json("user_123"))
    
    # Run cleanup (in production, use a scheduled job)
    deleted = policy.cleanup_old_conversations()
    print(f"\nCleaned up {deleted} old messages (older than 30 days)")
    
    # Handle user deletion request (GDPR right to erasure)
    deleted_user = policy.delete_user_data("user_456")
    print(f"Deleted {deleted_user} messages for user_456")
    
    # Show final stats
    stats = policy.get_stats()
    print(f"\nFinal stats: {stats}")

In [ ]:
# From: pii_filter.py

# From: AI Agents Book - Chapter 13, Section 13.7
# File: pii_filter.py

import re


class PIIFilter:
    """Filter personally identifiable information from messages."""
    
    def __init__(self):
        # Patterns for common PII
        self.patterns = {
            'email': r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b',
            'phone': r'\b\d{3}[-.]?\d{3}[-.]?\d{4}\b',
            'ssn': r'\b\d{3}-\d{2}-\d{4}\b',
            'credit_card': r'\b\d{4}[-\s]?\d{4}[-\s]?\d{4}[-\s]?\d{4}\b',
        }
    
    def filter_message(self, text: str) -> str:
        """Replace PII with placeholders."""
        filtered = text
        
        # Replace email addresses
        filtered = re.sub(self.patterns['email'], '[EMAIL]', filtered)
        
        # Replace phone numbers
        filtered = re.sub(self.patterns['phone'], '[PHONE]', filtered)
        
        # Replace SSNs
        filtered = re.sub(self.patterns['ssn'], '[SSN]', filtered)
        
        # Replace credit cards
        filtered = re.sub(self.patterns['credit_card'], '[CARD]', filtered)
        
        return filtered
    
    def contains_pii(self, text: str) -> bool:
        """Check if text contains any PII."""
        for pattern in self.patterns.values():
            if re.search(pattern, text):
                return True
        return False


def safe_add_message(memory, user_input, pii_filter):
    """Add message to memory after filtering PII."""
    if pii_filter.contains_pii(user_input):
        print("Warning: PII detected and filtered")
        user_input = pii_filter.filter_message(user_input)
    
    # Now safe to store
    memory.add_message(user_input)


# Usage
if __name__ == "__main__":
    filter = PIIFilter()
    
    test_message = "My email is john@example.com and phone is 555-123-4567"
    filtered = filter.filter_message(test_message)
    print(f"Original: {test_message}")
    print(f"Filtered: {filtered}")
    # Output: "My email is [EMAIL] and phone is [PHONE]"
    
    # Test detection
    print(f"Contains PII: {filter.contains_pii(test_message)}")
    print(f"Contains PII after filter: {filter.contains_pii(filtered)}")


In [ ]:
# From: time_based_cleanup.py

# From: AI Agents Book - Chapter 13, Section 13.7
# File: time_based_cleanup.py

from datetime import datetime, timedelta


class TimeBasedCleanup:
    """Delete conversations older than N days."""
    
    def __init__(self, max_age_days=30):
        self.max_age_days = max_age_days
        self.conversations = {}  # session_id -> (timestamp, messages)
    
    def add_conversation(self, session_id, messages):
        """Add a conversation with current timestamp."""
        self.conversations[session_id] = (datetime.now(), messages)
    
    def cleanup(self):
        """Remove conversations older than max_age_days."""
        cutoff = datetime.now() - timedelta(days=self.max_age_days)
        
        old_sessions = [
            sid for sid, (timestamp, _) in self.conversations.items()
            if timestamp < cutoff
        ]
        
        for sid in old_sessions:
            del self.conversations[sid]
        
        return len(old_sessions)
    
    def get_conversation(self, session_id):
        """Get conversation if it exists."""
        if session_id in self.conversations:
            return self.conversations[session_id][1]
        return None


# Usage
if __name__ == "__main__":
    cleanup = TimeBasedCleanup(max_age_days=7)
    
    # Add some test conversations
    cleanup.add_conversation("user_1", ["Hello", "Hi there!"])
    cleanup.add_conversation("user_2", ["What's the weather?", "It's sunny!"])
    
    # Simulate cleanup
    deleted = cleanup.cleanup()
    print(f"Removed {deleted} old conversations")
    print(f"Remaining: {len(cleanup.conversations)} conversations")


In [ ]:
# From: size_based_cleanup.py

# From: AI Agents Book - Chapter 13, Section 13.7
# File: size_based_cleanup.py

from collections import deque


class SizeBasedMemory:
    """Keep only the N most recent conversations per user."""
    
    def __init__(self, max_conversations_per_user=10):
        self.max_conversations = max_conversations_per_user
        self.user_conversations = {}  # user_id -> deque of conversations
    
    def add_conversation(self, user_id, conversation):
        """Add conversation, automatically dropping oldest when full."""
        if user_id not in self.user_conversations:
            self.user_conversations[user_id] = deque(maxlen=self.max_conversations)
        
        # Automatically drops oldest when full
        self.user_conversations[user_id].append(conversation)
    
    def get_recent_conversations(self, user_id, n=5):
        """Return n most recent conversations for user."""
        if user_id not in self.user_conversations:
            return []
        
        # Return n most recent
        return list(self.user_conversations[user_id])[-n:]
    
    def get_all_conversations(self, user_id):
        """Get all stored conversations for user."""
        if user_id not in self.user_conversations:
            return []
        return list(self.user_conversations[user_id])
    
    def get_conversation_count(self, user_id):
        """Get number of stored conversations for user."""
        if user_id not in self.user_conversations:
            return 0
        return len(self.user_conversations[user_id])


# Usage
if __name__ == "__main__":
    memory = SizeBasedMemory(max_conversations_per_user=3)
    
    # Add conversations for user
    for i in range(5):
        memory.add_conversation("user_123", {"id": i, "messages": [f"Conversation {i}"]})
    
    # Only 3 most recent are kept
    conversations = memory.get_all_conversations("user_123")
    print(f"Stored {len(conversations)} conversations (max 3)")
    for conv in conversations:
        print(f"  - {conv}")


In [ ]:
# From: importance_based_cleanup.py

# From: AI Agents Book - Chapter 13, Section 13.7
# File: importance_based_cleanup.py


class ImportanceBasedMemory:
    """Keep important conversations, discard routine ones."""
    
    def __init__(self):
        self.conversations = {}
    
    def add_conversation(self, conv_id, conversation):
        """Add a conversation."""
        self.conversations[conv_id] = conversation
    
    def calculate_importance(self, conversation):
        """Score conversation importance based on multiple factors."""
        score = 0
        
        # Long conversations are more important
        score += min(len(conversation.get("messages", [])), 10)
        
        # User-marked favorites
        if conversation.get("is_favorite"):
            score += 20
        
        # Contains entities (people, projects mentioned)
        score += len(conversation.get("entities", [])) * 2
        
        # Tool usage indicates complex task
        score += len(conversation.get("tool_calls", [])) * 3
        
        # Has summary (indicates substantial conversation)
        if conversation.get("summary"):
            score += 5
        
        return score
    
    def cleanup_low_importance(self, threshold=5):
        """Remove conversations below importance threshold."""
        to_delete = []
        
        for conv_id, conv in self.conversations.items():
            if self.calculate_importance(conv) < threshold:
                to_delete.append(conv_id)
        
        for conv_id in to_delete:
            del self.conversations[conv_id]
        
        return len(to_delete)
    
    def get_important_conversations(self, min_importance=10):
        """Get conversations above importance threshold."""
        return {
            conv_id: conv 
            for conv_id, conv in self.conversations.items()
            if self.calculate_importance(conv) >= min_importance
        }


# Usage
if __name__ == "__main__":
    memory = ImportanceBasedMemory()
    
    # Add conversations with different importance levels
    memory.add_conversation("conv_1", {
        "messages": ["hi", "bye"],
        "entities": [],
        "tool_calls": []
    })
    
    memory.add_conversation("conv_2", {
        "messages": ["Tell me about Project Alpha", "What's the status?", "Update the timeline"],
        "entities": ["Project Alpha", "Sarah"],
        "tool_calls": ["search", "calendar"],
        "is_favorite": True
    })
    
    # Show importance scores
    for conv_id, conv in memory.conversations.items():
        score = memory.calculate_importance(conv)
        print(f"{conv_id}: importance = {score}")
    
    # Cleanup low importance
    deleted = memory.cleanup_low_importance(threshold=5)
    print(f"\nDeleted {deleted} low-importance conversations")
    print(f"Remaining: {list(memory.conversations.keys())}")


In [ ]:
# From: partitioned_storage.py

# From: AI Agents Book - Chapter 13, Section 13.7
# File: partitioned_storage.py

from dotenv import load_dotenv
load_dotenv()

from datetime import datetime
from langchain_community.chat_message_histories import SQLChatMessageHistory


class PartitionedStorage:
    """Partition conversation data by date for large-scale applications."""
    
    def __init__(self, base_path="chat_history"):
        self.base_path = base_path
    
    def get_db_path(self, date=None):
        """Get database path for specific date partition."""
        if date is None:
            date = datetime.now()
        
        year_month = date.strftime("%Y_%m")
        return f"{self.base_path}_{year_month}.db"
    
    def get_session_history(self, session_id, date=None):
        """Get history from appropriate partition."""
        db_path = self.get_db_path(date)
        return SQLChatMessageHistory(
            session_id=session_id,
            connection=f"sqlite:///{db_path}"
        )
    
    def list_partitions(self):
        """List all existing partitions."""
        import glob
        return glob.glob(f"{self.base_path}_*.db")


# Usage
if __name__ == "__main__":
    storage = PartitionedStorage()
    
    # Current month
    current_history = storage.get_session_history("user_123")
    print(f"Current DB: {storage.get_db_path()}")
    
    # Specific month (for historical queries)
    jan_2024 = datetime(2024, 1, 1)
    old_db = storage.get_db_path(date=jan_2024)
    print(f"January 2024 DB: {old_db}")
    
    # List all partitions
    print(f"Existing partitions: {storage.list_partitions()}")


In [ ]:
# From: memory_cache.py

# From: AI Agents Book - Chapter 13, Section 13.7
# File: memory_cache.py

from datetime import datetime, timedelta


class CachedMemoryStore:
    """Cache frequently accessed conversations for performance."""
    
    def __init__(self, cache_duration_minutes=15):
        self.cache_duration = timedelta(minutes=cache_duration_minutes)
        self.cache = {}  # session_id -> (timestamp, messages)
        self.hits = 0
        self.misses = 0
    
    def get_messages(self, session_id):
        """Get messages with caching."""
        # Check cache first
        if session_id in self.cache:
            timestamp, messages = self.cache[session_id]
            if datetime.now() - timestamp < self.cache_duration:
                self.hits += 1
                return messages
        
        # Cache miss - load from database
        self.misses += 1
        messages = self._load_from_db(session_id)
        self.cache[session_id] = (datetime.now(), messages)
        return messages
    
    def _load_from_db(self, session_id):
        """Load from actual storage (implement per your backend)."""
        # Placeholder - implement actual database loading
        return []
    
    def invalidate_cache(self, session_id):
        """Clear cache for a session."""
        if session_id in self.cache:
            del self.cache[session_id]
    
    def clear_expired(self):
        """Remove expired cache entries."""
        now = datetime.now()
        expired = [
            sid for sid, (timestamp, _) in self.cache.items()
            if now - timestamp >= self.cache_duration
        ]
        for sid in expired:
            del self.cache[sid]
        return len(expired)
    
    def get_stats(self):
        """Get cache statistics."""
        total = self.hits + self.misses
        hit_rate = self.hits / total if total > 0 else 0
        return {
            "hits": self.hits,
            "misses": self.misses,
            "hit_rate": f"{hit_rate:.2%}",
            "cached_sessions": len(self.cache)
        }


# Usage
if __name__ == "__main__":
    cache = CachedMemoryStore(cache_duration_minutes=15)
    
    # Simulate access pattern
    for _ in range(5):
        cache.get_messages("user_123")  # First is miss, rest are hits
    
    cache.get_messages("user_456")  # New user, cache miss
    
    print("Cache stats:", cache.get_stats())


In [ ]:
# From: lazy_loading.py

# From: AI Agents Book - Chapter 13, Section 13.7
# File: lazy_loading.py

from dotenv import load_dotenv
load_dotenv()

from langchain_community.chat_message_histories import SQLChatMessageHistory


class LazyLoadMemory:
    """Don't load entire conversation history upfront."""
    
    def __init__(self, session_id, db_path):
        self.session_id = session_id
        self.db_path = db_path
        self._messages = None  # Not loaded yet
        self._loaded = False
    
    @property
    def messages(self):
        """Load messages only when accessed."""
        if self._messages is None:
            self._messages = self._load_messages()
            self._loaded = True
        return self._messages
    
    def _load_messages(self):
        """Load from database."""
        history = SQLChatMessageHistory(
            session_id=self.session_id,
            connection=self.db_path
        )
        return history.messages
    
    def get_recent(self, n=10):
        """Get only recent messages without loading all."""
        # For efficiency, could implement a direct DB query
        # This is a simplified version
        return self.messages[-n:] if self.messages else []
    
    def is_loaded(self):
        """Check if messages have been loaded."""
        return self._loaded
    
    def reload(self):
        """Force reload from database."""
        self._messages = None
        self._loaded = False
        return self.messages


# Usage
if __name__ == "__main__":
    memory = LazyLoadMemory("user_123", "sqlite:///chat.db")
    
    print(f"Loaded: {memory.is_loaded()}")  # False
    
    # Messages loaded on first access
    recent = memory.get_recent(5)
    print(f"Loaded: {memory.is_loaded()}")  # True
    print(f"Recent messages: {len(recent)}")


In [ ]:
# From: batch_operations.py

# From: AI Agents Book - Chapter 13, Section 13.7
# File: batch_operations.py

import sqlite3
from datetime import datetime, timedelta


class BatchMemoryProcessor:
    """Process multiple conversations efficiently with batch operations.
    
    Note: This assumes a message_store table with columns:
    - session_id, message_type, content, timestamp
    
    See retention_policy.py or production_memory_manager.py for
    the table creation schema.
    """
    
    def __init__(self, db_path):
        self.db_path = db_path
        self._init_db()
    
    def _init_db(self):
        """Initialize database with required schema."""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS message_store (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                session_id TEXT NOT NULL,
                message_type TEXT NOT NULL,
                content TEXT NOT NULL,
                timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
            )
        """)
        
        cursor.execute("""
            CREATE INDEX IF NOT EXISTS idx_session_id 
            ON message_store(session_id)
        """)
        cursor.execute("""
            CREATE INDEX IF NOT EXISTS idx_timestamp 
            ON message_store(timestamp)
        """)
        
        conn.commit()
        conn.close()
    
    def add_message(self, session_id: str, message_type: str, content: str, 
                    timestamp: datetime = None):
        """Add a message (for testing purposes)."""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute("""
            INSERT INTO message_store (session_id, message_type, content, timestamp)
            VALUES (?, ?, ?, ?)
        """, (session_id, message_type, content, timestamp or datetime.now()))
        
        conn.commit()
        conn.close()
    
    def batch_cleanup(self, session_ids, cutoff_date):
        """Delete old messages for multiple users in one query."""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        # Use IN clause for batch operation
        placeholders = ','.join('?' * len(session_ids))
        cursor.execute(f"""
            DELETE FROM message_store 
            WHERE session_id IN ({placeholders})
            AND timestamp < ?
        """, (*session_ids, cutoff_date))
        
        deleted = cursor.rowcount
        conn.commit()
        conn.close()
        
        return deleted
    
    def batch_export(self, session_ids):
        """Export data for multiple users efficiently."""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        placeholders = ','.join('?' * len(session_ids))
        cursor.execute(f"""
            SELECT session_id, message_type, content, timestamp 
            FROM message_store 
            WHERE session_id IN ({placeholders})
            ORDER BY session_id, timestamp
        """, session_ids)
        
        results = cursor.fetchall()
        conn.close()
        
        # Group by session_id
        exports = {}
        for session_id, msg_type, content, timestamp in results:
            if session_id not in exports:
                exports[session_id] = []
            exports[session_id].append({
                "type": msg_type,
                "content": content,
                "timestamp": str(timestamp)
            })
        
        return exports
    
    def batch_delete_users(self, session_ids):
        """Delete all data for multiple users (GDPR bulk request)."""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        placeholders = ','.join('?' * len(session_ids))
        cursor.execute(f"""
            DELETE FROM message_store 
            WHERE session_id IN ({placeholders})
        """, session_ids)
        
        deleted = cursor.rowcount
        conn.commit()
        conn.close()
        
        return deleted
    
    def get_stats(self):
        """Get database statistics."""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute("SELECT COUNT(*) FROM message_store")
        total = cursor.fetchone()[0]
        
        cursor.execute("SELECT COUNT(DISTINCT session_id) FROM message_store")
        sessions = cursor.fetchone()[0]
        
        conn.close()
        return {"total_messages": total, "total_sessions": sessions}


# Usage
if __name__ == "__main__":
    import os
    
    # Use a fresh test database
    test_db = "batch_test.db"
    if os.path.exists(test_db):
        os.remove(test_db)
    
    processor = BatchMemoryProcessor(test_db)
    
    # Add sample data - some old, some new
    old_date = datetime.now() - timedelta(days=60)
    recent_date = datetime.now() - timedelta(days=5)
    
    print("Adding sample messages...")
    
    # Old messages (will be cleaned up)
    processor.add_message("user_1", "human", "Old message 1", old_date)
    processor.add_message("user_1", "ai", "Old response 1", old_date)
    processor.add_message("user_2", "human", "Old message 2", old_date)
    
    # Recent messages (will be kept)
    processor.add_message("user_1", "human", "Recent message", recent_date)
    processor.add_message("user_2", "human", "Recent message", recent_date)
    processor.add_message("user_3", "human", "User 3 message", recent_date)
    
    print(f"Stats before cleanup: {processor.get_stats()}")
    
    # Batch cleanup for multiple users (messages older than 30 days)
    users_to_clean = ["user_1", "user_2", "user_3"]
    cutoff = datetime.now() - timedelta(days=30)
    
    deleted = processor.batch_cleanup(users_to_clean, cutoff)
    print(f"\nBatch cleanup: deleted {deleted} old messages")
    print(f"Stats after cleanup: {processor.get_stats()}")
    
    # Batch export
    print("\nBatch export:")
    exports = processor.batch_export(["user_1", "user_2"])
    for user_id, messages in exports.items():
        print(f"  {user_id}: {len(messages)} messages")
    
    # Batch delete (GDPR)
    deleted = processor.batch_delete_users(["user_3"])
    print(f"\nBatch delete user_3: {deleted} messages deleted")
    print(f"Final stats: {processor.get_stats()}")
    
    # Cleanup test file
    os.remove(test_db)
    print("\nTest complete!")

In [ ]:
# From: cost_optimization.py

# From: AI Agents Book - Chapter 13, Section 13.7
# File: cost_optimization.py


class CostOptimizedVectorMemory:
    """Only create embeddings for important messages to save costs."""
    
    def __init__(self, collection, important_threshold=0.7):
        self.collection = collection
        self.important_threshold = important_threshold
        self.vectorized_count = 0
        self.skipped_count = 0
    
    def should_store_vector(self, message):
        """Decide if message is important enough to vectorize."""
        # Skip very short messages
        if len(message) < 10:
            return False
        
        # Skip routine responses
        routine_phrases = ["ok", "thanks", "bye", "hello", "hi", "yes", "no"]
        if message.lower().strip() in routine_phrases:
            return False
        
        # Skip if mostly punctuation
        alpha_ratio = sum(c.isalpha() for c in message) / max(len(message), 1)
        if alpha_ratio < 0.5:
            return False
        
        return True
    
    def add_message(self, message, metadata=None):
        """Only create embeddings for important messages."""
        if self.should_store_vector(message):
            # Create embedding and store (costs money)
            self.collection.add(
                documents=[message],
                metadatas=[metadata or {}],
                ids=[f"msg_{self.vectorized_count}"]
            )
            self.vectorized_count += 1
            return True
        else:
            # Skip - store in cheaper text-only storage if needed
            self.skipped_count += 1
            return False
    
    def get_stats(self):
        """Get cost optimization statistics."""
        total = self.vectorized_count + self.skipped_count
        savings = self.skipped_count / total if total > 0 else 0
        return {
            "vectorized": self.vectorized_count,
            "skipped": self.skipped_count,
            "savings_rate": f"{savings:.2%}"
        }


# Usage
if __name__ == "__main__":
    # Mock collection for demo
    class MockCollection:
        def add(self, documents, metadatas, ids):
            print(f"  Stored: {documents[0][:50]}...")
    
    memory = CostOptimizedVectorMemory(MockCollection())
    
    test_messages = [
        "hi",  # Skip
        "thanks",  # Skip
        "Can you explain how machine learning algorithms work?",  # Store
        "ok",  # Skip
        "I need help planning my project timeline for Q2",  # Store
        "bye",  # Skip
    ]
    
    print("Processing messages:")
    for msg in test_messages:
        stored = memory.add_message(msg)
        status = "STORED" if stored else "SKIPPED"
        print(f"  [{status}] {msg[:40]}...")
    
    print(f"\nStats: {memory.get_stats()}")


In [ ]:
# From: tiered_storage.py

# From: AI Agents Book - Chapter 13, Section 13.7
# File: tiered_storage.py

from datetime import datetime


class TieredStorage:
    """Move old conversations to cheaper storage tiers.
    
    Tiers:
    - Hot (< 7 days): Fast access, e.g., Redis
    - Warm (7-30 days): Moderate speed, e.g., SQLite
    - Cold (> 30 days): Slow but cheap, e.g., S3
    """
    
    def __init__(self):
        self.hot_storage = {}   # Recent - in-memory or Redis
        self.warm_storage = {}  # 7-30 days - SQLite
        self.cold_storage = {}  # 30+ days - S3 or similar
    
    def get_tier(self, message_date):
        """Determine storage tier based on message age."""
        age_days = (datetime.now() - message_date).days
        
        if age_days < 7:
            return "hot"
        elif age_days < 30:
            return "warm"
        else:
            return "cold"
    
    def store_message(self, session_id, message, timestamp=None):
        """Store message in appropriate tier."""
        if timestamp is None:
            timestamp = datetime.now()
        
        tier = self.get_tier(timestamp)
        
        if tier == "hot":
            self._store_hot(session_id, message, timestamp)
        elif tier == "warm":
            self._store_warm(session_id, message, timestamp)
        else:
            self._store_cold(session_id, message, timestamp)
    
    def get_messages(self, session_id, message_date=None):
        """Route to appropriate storage tier."""
        if message_date is None:
            # Get from all tiers
            messages = []
            messages.extend(self._get_from_hot(session_id))
            messages.extend(self._get_from_warm(session_id))
            messages.extend(self._get_from_cold(session_id))
            return messages
        
        tier = self.get_tier(message_date)
        
        if tier == "hot":
            return self._get_from_hot(session_id)
        elif tier == "warm":
            return self._get_from_warm(session_id)
        else:
            return self._get_from_cold(session_id)
    
    def migrate_to_colder_tier(self):
        """Move aged data to appropriate tier (run periodically)."""
        moved = {"hot_to_warm": 0, "warm_to_cold": 0}
        
        # Move from hot to warm
        for session_id, messages in list(self.hot_storage.items()):
            for msg in messages[:]:
                if self.get_tier(msg["timestamp"]) == "warm":
                    self._store_warm(session_id, msg["content"], msg["timestamp"])
                    messages.remove(msg)
                    moved["hot_to_warm"] += 1
        
        # Move from warm to cold
        for session_id, messages in list(self.warm_storage.items()):
            for msg in messages[:]:
                if self.get_tier(msg["timestamp"]) == "cold":
                    self._store_cold(session_id, msg["content"], msg["timestamp"])
                    messages.remove(msg)
                    moved["warm_to_cold"] += 1
        
        return moved
    
    # Tier-specific implementations
    def _store_hot(self, session_id, message, timestamp):
        if session_id not in self.hot_storage:
            self.hot_storage[session_id] = []
        self.hot_storage[session_id].append({"content": message, "timestamp": timestamp})
    
    def _store_warm(self, session_id, message, timestamp):
        if session_id not in self.warm_storage:
            self.warm_storage[session_id] = []
        self.warm_storage[session_id].append({"content": message, "timestamp": timestamp})
    
    def _store_cold(self, session_id, message, timestamp):
        if session_id not in self.cold_storage:
            self.cold_storage[session_id] = []
        self.cold_storage[session_id].append({"content": message, "timestamp": timestamp})
    
    def _get_from_hot(self, session_id):
        return self.hot_storage.get(session_id, [])
    
    def _get_from_warm(self, session_id):
        return self.warm_storage.get(session_id, [])
    
    def _get_from_cold(self, session_id):
        return self.cold_storage.get(session_id, [])


# Usage
if __name__ == "__main__":
    storage = TieredStorage()
    
    # Store recent message (goes to hot)
    storage.store_message("user_123", "Hello!")
    
    # Store older message (simulated)
    from datetime import timedelta
    old_date = datetime.now() - timedelta(days=15)
    storage.store_message("user_123", "Old message", old_date)
    
    very_old_date = datetime.now() - timedelta(days=45)
    storage.store_message("user_123", "Very old message", very_old_date)
    
    print(f"Hot storage: {len(storage.hot_storage.get('user_123', []))} messages")
    print(f"Warm storage: {len(storage.warm_storage.get('user_123', []))} messages")
    print(f"Cold storage: {len(storage.cold_storage.get('user_123', []))} messages")


In [ ]:
# From: memory_monitoring.py

# From: AI Agents Book - Chapter 13, Section 13.7
# File: memory_monitoring.py

from datetime import datetime


class MemoryMonitor:
    """Track memory system health and performance metrics."""
    
    def __init__(self):
        self.metrics = {
            "total_conversations": 0,
            "total_messages": 0,
            "avg_conversation_length": 0,
            "cache_hits": 0,
            "cache_misses": 0,
            "storage_operations": 0,
            "errors": 0,
        }
        self.start_time = datetime.now()
    
    def log_conversation(self, message_count):
        """Log a conversation."""
        self.metrics["total_conversations"] += 1
        self.metrics["total_messages"] += message_count
        self._update_average()
    
    def _update_average(self):
        """Update average conversation length."""
        if self.metrics["total_conversations"] > 0:
            self.metrics["avg_conversation_length"] = (
                self.metrics["total_messages"] / self.metrics["total_conversations"]
            )
    
    def log_access(self, session_id, is_cache_hit):
        """Log memory access pattern."""
        if is_cache_hit:
            self.metrics["cache_hits"] += 1
        else:
            self.metrics["cache_misses"] += 1
    
    def log_storage_operation(self):
        """Log a storage operation."""
        self.metrics["storage_operations"] += 1
    
    def log_error(self):
        """Log an error."""
        self.metrics["errors"] += 1
    
    def get_cache_hit_rate(self):
        """Calculate cache effectiveness."""
        total = self.metrics["cache_hits"] + self.metrics["cache_misses"]
        if total == 0:
            return 0
        return self.metrics["cache_hits"] / total
    
    def get_uptime(self):
        """Get time since monitoring started."""
        return datetime.now() - self.start_time
    
    def get_report(self):
        """Generate memory system report."""
        return {
            **self.metrics,
            "cache_hit_rate": f"{self.get_cache_hit_rate():.2%}",
            "uptime": str(self.get_uptime()),
            "timestamp": datetime.now().isoformat(),
        }
    
    def reset(self):
        """Reset all metrics."""
        self.__init__()


# Usage
if __name__ == "__main__":
    monitor = MemoryMonitor()
    
    # Simulate activity
    monitor.log_conversation(message_count=10)
    monitor.log_conversation(message_count=5)
    monitor.log_access("user_123", is_cache_hit=True)
    monitor.log_access("user_456", is_cache_hit=False)
    monitor.log_access("user_123", is_cache_hit=True)
    monitor.log_storage_operation()
    
    print("Memory System Report:")
    report = monitor.get_report()
    for key, value in report.items():
        print(f"  {key}: {value}")


In [ ]:
# From: production_memory_manager.py

# From: AI Agents Book - Chapter 13, Section 13.7
# File: production_memory_manager.py

from dotenv import load_dotenv
load_dotenv()

from datetime import datetime, timedelta
import sqlite3
import json
from langchain_core.messages import HumanMessage, AIMessage

# Import from other files in this section
from pii_filter import PIIFilter
from memory_monitoring import MemoryMonitor


class ProductionMemoryManager:
    """Complete production-ready memory manager.
    
    Combines:
    - PII filtering
    - Retention policies with timestamps
    - Monitoring
    - GDPR compliance (export/delete)
    
    Note: This uses a custom schema with timestamps rather than
    the default SQLChatMessageHistory which lacks timestamp support.
    """
    
    def __init__(self, db_path="prod_memory.db", retention_days=30):
        self.db_path = db_path
        self.retention_days = retention_days
        self.pii_filter = PIIFilter()
        self.monitor = MemoryMonitor()
        self._init_db()
    
    def _init_db(self):
        """Initialize database with timestamp-enabled schema."""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        # Create table with timestamp column
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS message_store (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                session_id TEXT NOT NULL,
                message_type TEXT NOT NULL,
                content TEXT NOT NULL,
                timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
            )
        """)
        
        # Create indexes for efficient queries
        cursor.execute("""
            CREATE INDEX IF NOT EXISTS idx_session_id 
            ON message_store(session_id)
        """)
        cursor.execute("""
            CREATE INDEX IF NOT EXISTS idx_timestamp 
            ON message_store(timestamp)
        """)
        
        conn.commit()
        conn.close()
    
    def get_session_history(self, session_id: str):
        """Get message history for a session."""
        self.monitor.log_access(session_id, is_cache_hit=False)
        
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute("""
            SELECT message_type, content FROM message_store
            WHERE session_id = ?
            ORDER BY timestamp
        """, (session_id,))
        
        messages = []
        for msg_type, content in cursor.fetchall():
            if msg_type == "human":
                messages.append(HumanMessage(content=content))
            else:
                messages.append(AIMessage(content=content))
        
        conn.close()
        return messages
    
    def add_user_message(self, session_id: str, message: str):
        """Add message with PII filtering."""
        # Filter PII before storing
        filtered_message = self.pii_filter.filter_message(message)
        
        if filtered_message != message:
            print(f"[INFO] PII detected and filtered for session {session_id}")
        
        self._store_message(session_id, "human", filtered_message)
        self.monitor.log_storage_operation()
    
    def add_ai_message(self, session_id: str, message: str):
        """Add AI response to history."""
        self._store_message(session_id, "ai", message)
        self.monitor.log_storage_operation()
    
    def _store_message(self, session_id: str, message_type: str, content: str):
        """Store a message with timestamp."""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute("""
            INSERT INTO message_store (session_id, message_type, content, timestamp)
            VALUES (?, ?, ?, ?)
        """, (session_id, message_type, content, datetime.now()))
        
        conn.commit()
        conn.close()
    
    def run_maintenance(self):
        """Run periodic maintenance tasks."""
        # Clean old data
        cutoff = datetime.now() - timedelta(days=self.retention_days)
        deleted = self._cleanup_old_data(cutoff)
        
        # Get metrics
        report = self.monitor.get_report()
        
        return {
            "deleted_messages": deleted,
            "metrics": report,
            "timestamp": datetime.now().isoformat()
        }
    
    def _cleanup_old_data(self, cutoff_date):
        """Delete messages older than cutoff date."""
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            cursor.execute(
                "DELETE FROM message_store WHERE timestamp < ?", 
                (cutoff_date,)
            )
            deleted = cursor.rowcount
            conn.commit()
            conn.close()
            return deleted
        except Exception as e:
            self.monitor.log_error()
            print(f"[ERROR] Cleanup failed: {e}")
            return 0
    
    def export_user_data(self, session_id: str):
        """Export all user data (GDPR compliance)."""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute("""
            SELECT message_type, content, timestamp FROM message_store
            WHERE session_id = ?
            ORDER BY timestamp
        """, (session_id,))
        
        messages = [
            {"type": row[0], "content": row[1], "timestamp": str(row[2])}
            for row in cursor.fetchall()
        ]
        
        conn.close()
        
        return {
            "session_id": session_id,
            "messages": messages,
            "export_date": datetime.now().isoformat()
        }
    
    def delete_user_data(self, session_id: str):
        """Delete all user data (GDPR right to erasure)."""
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            cursor.execute(
                "DELETE FROM message_store WHERE session_id = ?", 
                (session_id,)
            )
            deleted = cursor.rowcount
            conn.commit()
            conn.close()
            return deleted
        except Exception as e:
            self.monitor.log_error()
            print(f"[ERROR] Delete failed: {e}")
            return 0
    
    def get_metrics(self):
        """Get current system metrics."""
        return self.monitor.get_report()
    
    def get_stats(self):
        """Get database statistics."""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute("SELECT COUNT(*) FROM message_store")
        total_messages = cursor.fetchone()[0]
        
        cursor.execute("SELECT COUNT(DISTINCT session_id) FROM message_store")
        total_sessions = cursor.fetchone()[0]
        
        conn.close()
        
        return {
            "total_messages": total_messages,
            "total_sessions": total_sessions
        }


# Usage
if __name__ == "__main__":
    manager = ProductionMemoryManager(retention_days=30)
    
    # Normal operation - PII is filtered
    print("Adding messages with PII filtering...")
    manager.add_user_message(
        "user_123", 
        "My email is sensitive@example.com and I need help"
    )
    manager.add_ai_message(
        "user_123",
        "I'd be happy to help! What do you need?"
    )
    
    # Add another user
    manager.add_user_message("user_456", "Hello there!")
    manager.add_ai_message("user_456", "Hi! How can I assist you?")
    
    # Check stats
    print("\nDatabase stats:")
    stats = manager.get_stats()
    for key, value in stats.items():
        print(f"  {key}: {value}")
    
    # Check metrics
    print("\nSystem metrics:")
    metrics = manager.get_metrics()
    for key, value in metrics.items():
        print(f"  {key}: {value}")
    
    # Periodic maintenance (run daily via cron)
    report = manager.run_maintenance()
    print(f"\nMaintenance: deleted {report['deleted_messages']} old messages")
    
    # Handle GDPR requests
    print("\n--- GDPR Export ---")
    user_data = manager.export_user_data("user_123")
    print(f"Exported {len(user_data['messages'])} messages for user_123")
    print(json.dumps(user_data, indent=2))
    
    print("\n--- GDPR Delete ---")
    deleted = manager.delete_user_data("user_456")
    print(f"Deleted {deleted} messages for user_456")
    
    # Final stats
    print("\nFinal stats:")
    stats = manager.get_stats()
    for key, value in stats.items():
        print(f"  {key}: {value}")

In [ ]:
# From: personal_assistant_challenge.py

# From: AI Agents Book - Chapter 13
# File: personal_assistant_challenge.py
# Chapter 13 Challenge: Personal AI Assistant

from dotenv import load_dotenv
load_dotenv()

"""
Chapter 13 Challenge: Build a Complete Personal AI Assistant

Requirements:
1. Remember conversations across sessions (persistence)
2. Track entities mentioned in conversations (people, projects, preferences)
3. Use tools (at least 2: calculator and weather/time)
4. Implement semantic search to recall relevant past conversations
5. Auto-summarize when conversations get long (>500 tokens)
6. Handle privacy with PII filtering
7. Implement cleanup with 30-day retention policy
8. Support multiple users with isolated memories

Bonus Challenges:
- Add a "remember" command for explicit fact storage
- Implement importance-based cleanup (keep important convos longer)
- Add conversation export feature (GDPR compliance)
- Build a simple CLI or web interface
- Track and report memory system metrics

Time Estimate: 3-5 hours
"""

from langchain_openai import ChatOpenAI
from langchain_classic.agents import create_react_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
# import chromadb  # Uncomment when implementing semantic search


class PersonalAssistant:
    """Your personal AI assistant with production-ready memory."""
    
    def __init__(self, user_id):
        self.user_id = user_id
        self.setup_memory()
        self.setup_agent()
    
    def setup_memory(self):
        """Set up conversation memory, entity memory, vector memory."""
        # TODO: Initialize conversation memory (SQLite-backed)
        # TODO: Initialize entity memory
        # TODO: Initialize vector memory (ChromaDB)
        # TODO: Initialize PII filter
        # TODO: Set up retention policy
        pass
    
    def setup_agent(self):
        """Create agent with tools and memory."""
        # TODO: Define tools (calculator, weather, etc.)
        # TODO: Create prompt with memory context
        # TODO: Set up agent with summarization
        pass
    
    def chat(self, message):
        """Process message, use tools, store in memory."""
        # TODO: Filter PII from input
        # TODO: Retrieve relevant memories
        # TODO: Get entity context
        # TODO: Run agent
        # TODO: Store conversation in memory
        # TODO: Extract and store entities
        # TODO: Check if summarization needed
        pass
    
    def remember(self, fact):
        """Store explicit fact in semantic memory."""
        # TODO: Add fact to vector database
        pass
    
    def recall(self, query):
        """Search relevant past conversations."""
        # TODO: Semantic search in vector database
        # TODO: Return relevant memories
        pass
    
    def export_data(self):
        """Export all user data (GDPR compliance)."""
        # TODO: Gather all user data
        # TODO: Format for export
        pass
    
    def delete_data(self):
        """Delete all user data (GDPR right to erasure)."""
        # TODO: Delete from all memory systems
        pass
    
    def cleanup(self):
        """Run retention policy."""
        # TODO: Delete old conversations
        # TODO: Report metrics
        pass


# Define your tools here
@tool
def calculator(expression: str) -> str:
    """Evaluate a math expression like '2+2' or '10*5'."""
    try:
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {str(e)}"


@tool
def get_weather(city: str) -> str:
    """Get current weather for a city."""
    # Mock implementation - replace with real API
    weather_data = {
        "Seattle": "Rainy, 55°F",
        "NYC": "Sunny, 68°F",
        "Austin": "Hot, 95°F",
    }
    return weather_data.get(city, f"Weather in {city}: Partly cloudy, 70°F")


@tool
def get_time(timezone: str = "UTC") -> str:
    """Get current time in specified timezone."""
    from datetime import datetime
    # Simple implementation - enhance with pytz for real timezones
    return f"Current time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"


# Test your assistant
if __name__ == "__main__":
    print("=" * 60)
    print("Chapter 13 Challenge: Personal AI Assistant")
    print("=" * 60)
    print("\nThis is a starter template. Implement the TODO items!")
    print("\nExpected behavior when complete:")
    print("  1. assistant.chat() processes messages with memory")
    print("  2. assistant.remember() stores facts")
    print("  3. assistant.recall() searches past conversations")
    print("  4. PII is filtered automatically")
    print("  5. Old conversations are cleaned up")
    print("  6. Multiple users have isolated memories")
    print("\n" + "=" * 60)
    
    # Uncomment when implementing:
    # assistant = PersonalAssistant("user_123")
    # assistant.chat("My name is Alice and I'm a Python developer.")
    # assistant.chat("What's the weather in Seattle?")
    # assistant.chat("What did I tell you about myself?")
    # assistant.remember("Alice prefers dark mode")
    # print(assistant.recall("preferences"))


---
## Next Steps

- Check your answers in **chapter_13_agent_memory_solutions.ipynb**
- Proceed to **Chapter 14**